In [1]:
#!/usr/local/bin/python3
# -*- coding: utf-8 -*-

from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.dml import MSO_THEME_COLOR
from inspect import getmembers
from pprint import pprint
from pptx.parts.chart import ChartPart
from pptx.parts.embeddedpackage import EmbeddedXlsxPart
from pptx.chart.data import ChartData
import pandas as pd
import numpy as np
import copy
import six
import time
import datetime
import csv

In [2]:
# Path to files and data
path_to_template = './template.pptx'
path_to_table_data = './data/table_data_20194_2019-05-17.csv'
path_to_list_data = './data/list_data_20194_2019-05-17.csv'
path_to_chart_data = './data/charts_data_20194_2019-05-17.csv'
prs = Presentation(path_to_template)
encoding = 'utf-8'

In [3]:
# abbr month, month, month in the case
month = [
    ['мес', 'месяц', 'месяце'],
    ['янв', 'январь', 'январе'],
    ['фев', 'февраль', 'феврале'],
    ['мар', 'март', 'марте'],
    ['апр', 'апрель', 'апреле'],
    ['май', 'май', 'мае'],
    ['июн', 'июнь', 'июне'],
    ['июл', 'июль', 'июле'],
    ['авг', 'август', 'августе'],
    ['сен', 'сентябрь', 'сентябре'],
    ['окт', 'октябрь', 'октябре'],
    ['ноя', 'ноябрь', 'ноябре'],
    ['дек', 'декабрь', 'декабре']
]

In [4]:
# Get report month and previous mounth
today = datetime.date.today()
first_day = today.replace(day=1)
report_month = first_day - datetime.timedelta(days=1)
prev_first_day = report_month.replace(day=1)
report_month_prev = prev_first_day - datetime.timedelta(days=1)
prev2_firstday = report_month_prev.replace(day=1)
report_month_prev2 = prev2_firstday - datetime.timedelta(days=1)

In [5]:
# First slide: change month and year
first_slide = prs.slides[0]
date_shape = first_slide.shapes[3]
text_frame = date_shape.text_frame
p = text_frame.paragraphs[0]
p.text = month[report_month.month][1].upper() + ' ' + str(report_month.year)
font = p.font
font.name = "Verdana"
font.size = Pt(18)
font.color.rgb = RGBColor(0x70, 0x80, 0x90)

In [6]:
# Second slide: change month and year
# TODO: integrate the map generation
second_slide = prs.slides[1]
text_frame = second_slide.shapes[1].text_frame
text_frame.paragraphs[1].text = 'обращения, поступившие в '.upper() + str(
    month[report_month.month][2]).upper() + ' ' + str(report_month.year)
p = text_frame.paragraphs[1]
font = p.font
font.name = "Verdana"
font.size = Pt(12)
font.color.rgb = RGBColor(0x70, 0x80, 0x90)

text_frame = second_slide.shapes[3].text_frame
text_frame.paragraphs[1].text = str('за ' +
                                    month[report_month.month][1]).upper() + " " + str(report_month.year)
p = text_frame.paragraphs[1]
font = p.font
font.name = "Verdana"
font.size = Pt(10)
font.color.rgb = RGBColor(0x30, 0x30, 0x30)

In [7]:
# Copying regions slide
def _get_blank_slide_layout(pres):
    layout_items_count = [
        len(layout.placeholders) for layout in pres.slide_layouts
    ]
    min_items = min(layout_items_count)
    blank_layout_id = layout_items_count.index(min_items)
    return pres.slide_layouts[blank_layout_id]


def duplicate_slide(pres, index):
    """Duplicate the slide with the given index in pres.

    Adds slide to the end of the presentation"""
    source = pres.slides[index]
    blank_slide_layout = _get_blank_slide_layout(pres)
    dest = pres.slides.add_slide(blank_slide_layout)

    for shape in source.shapes:
        newel = copy.deepcopy(shape.element)
        dest.shapes._spTree.insert_element_before(newel, 'p:extLst')

    for key, value in source.part.rels.items():
        # Make sure we don't copy a notesSlide relation as that won't exist
        if "notesSlide" not in value.reltype:
            target = value._target
            # if the relationship was a chart, we need to duplicate the embedded chart part and xlsx
            if "chart" in value.reltype:
                partname = target.package.next_partname(
                    ChartPart.partname_template)
                xlsx_blob = target.chart_workbook.xlsx_part.blob
                target = ChartPart(
                    partname,
                    target.content_type,
                    copy.deepcopy(target._element),
                    package=target.package)
                target.chart_workbook.xlsx_part = EmbeddedXlsxPart.new(
                    xlsx_blob, target.package)
            dest.part.rels.add_relationship(value.reltype, target, value.rId)
    return dest


for i in range(84):
    duplicate_slide(prs, 2)

In [8]:
# Get data
table_data_file = pd.read_csv(path_to_table_data, sep=';', encoding=encoding)
chart_data_file = pd.read_csv(path_to_chart_data, sep=';', encoding=encoding)
list_data_file = pd.read_csv(path_to_list_data, sep=';', encoding=encoding)

In [9]:
# Get regions
regions = table_data_file['region_title'].unique()

In [10]:
# Fill in slides region name, appeals received in..., status, change image
for i in range(2, 87):
    regions_slide = prs.slides[i]
    p = regions_slide.shapes[1].text_frame.paragraphs[0]
    regions_slide.shapes[1].text_frame.paragraphs[0].text = str(
        regions[i-2]).upper()
    font = p.font
    font.name = "Verdana"
    font.size = Pt(12)
    font.bold = True
    font.color.rgb = RGBColor(0x20, 0x50, 0x80)

    p = regions_slide.shapes[1].text_frame.paragraphs[1]
    regions_slide.shapes[1].text_frame.paragraphs[1].text = 'обращения, поступившие в '.upper(
    ) + str(month[report_month.month][2]).upper() + ' ' + str(report_month.year)
    font = p.font
    font.name = "Verdana"
    font.size = Pt(12)
    font.color.rgb = RGBColor(0x70, 0x80, 0x90)

    p = regions_slide.shapes[3].text_frame.paragraphs[1]
    regions_slide.shapes[3].text_frame.paragraphs[1].text = 'по состоянию на '.upper(
    ) + str(month[report_month.month][1]).upper() + ' ' + str(report_month.year)
    font = p.font
    font.name = "Verdana"
    font.size = Pt(8)
    font.color.rgb = RGBColor(0x30, 0x30, 0x30)

    img_path = './img/' + str(i) + '.png'
    top = Pt(70)
    height = Pt(100)
    left = Pt(0)
    pic = regions_slide.shapes.add_picture(img_path, left, top, height)

In [11]:
# Add list
list_data = []
list_data_file = open(path_to_list_data, 'r', encoding=encoding)
with list_data_file:
    list_data_file.reader = csv.reader(list_data_file, delimiter=';')
    for row in list_data_file.reader:
        list_data.append(row)

lst = [
    'обращений', 'вопросов', 'организаций', 'наименований вопросов',
    'вопросов на 1 обращение', 'вопросов на 1000 жителей'
]

for i in range(1, 87):

    slide = prs.slides[i]
    shape = slide.shapes[11]
    text_frame = shape.text_frame
    text_frame.clear()

    for l in range(0, len(lst)):
        name = lst[l].upper()
        value = list_data[i][l + l + 1]
        value = value.replace(',', '.')
        delta = list_data[i][l + l + 2]
        delta = delta.replace(',', '.')
        if float(delta) > 0:
            delta = "+" + delta
        text = (name + ': ' + value + ' (' + delta + ')')
        if text_frame.text == '':
            text_frame.paragraphs[l].text = text
            run = text_frame.paragraphs[0]
            font = run.font
            font.color.rgb = RGBColor(0x3b, 0x38, 0x38)
            font.name = 'Calibri'
            font.size = Pt(10)
            font.bold = False
            font.italic = None
        else:
            run = text_frame.add_paragraph()
            run.text = text
            font = run.font
            font.color.rgb = RGBColor(0x3b, 0x38, 0x38)
            font.name = 'Calibri'
            font.size = Pt(10)
            font.bold = False
            font.italic = None

In [12]:
# Add chart data in chart shape
list_for_char = []
chart_data_file = open(path_to_chart_data, 'r', encoding=encoding)
with chart_data_file:
    chart_data_file.reader = csv.reader(chart_data_file, delimiter=';')
    for row in chart_data_file.reader:
        list_for_char.append(row)

month_prev2 = str(month[report_month_prev2.month][1]) + \
    " " + str(report_month_prev2.year)
month_prev = str(month[report_month_prev.month][1]) + \
    " " + str(report_month_prev.year)
report_month = str(month[report_month.month][1]) + " " + str(report_month.year)

for i in range(1, 87):
    slide = prs.slides[i]
    shape = slide.shapes[6]  # Status
    chart = shape.chart
    chart_data = ChartData()
    chart_data.categories = 'Рассмотрено', 'Находится на рассмотрении'
    x1 = int(list_for_char[i][1])
    x2 = int(list_for_char[i][2])
    chart_data.add_series('Количество вопросов', (x1, x2))
    chart.replace_data(chart_data)

    shape = slide.shapes[8]
    chart = shape.chart
    chart_data = ChartData()
    chart_data.categories = 'Федеральный', 'Региональный', 'Муниципальный'
    x3 = int(list_for_char[i][3])
    x4 = int(list_for_char[i][4])
    x5 = int(list_for_char[i][5])
    chart_data.add_series('Количество вопросов', (x3, x4, x5))
    chart.replace_data(chart_data)

    shape = slide.shapes[10]
    chart = shape.chart
    chart_data = ChartData()
    chart_data.categories = month_prev2.title(), month_prev.title(), report_month.title()
    x6 = int(list_for_char[i][6])  # treatments
    x7 = int(list_for_char[i][7])
    x8 = int(list_for_char[i][8])
    x9 = int(list_for_char[i][9])  # questions
    x10 = int(list_for_char[i][10])
    x11 = int(list_for_char[i][11])
    chart_data.add_series('Количество обращений', (x6, x7, x8))
    chart_data.add_series('Количество вопросов', (x9, x10, x11))
    chart.replace_data(chart_data)

In [13]:
title_text = []

slide = prs.slides[1]
shape = slide.shapes[1]
paragraph = shape.text_frame.paragraphs[0]
title_text.append(paragraph.text)


for n in range(2, 87):  # 87 = slides len
    slide = prs.slides[n]
    shape = slide.shapes[1]
    paragraph = shape.text_frame.paragraphs[0]
    for run in paragraph.runs:
        title_text.append(run.text)  

In [14]:
# Add top 10 question in table

top_10 = open(path_to_table_data, 'r', encoding=encoding)
list_top_10 = []

with top_10:
    top_10.reader = csv.reader(top_10, delimiter=';')
    for row in top_10.reader:
        list_top_10.append(row)

table_shapes = []

for slide in prs.slides:
    for idx, shape in enumerate(slide.shapes):
        if not shape.has_table:
            continue
        table_shapes.append(idx)

count_subject_count = 0
for i in range(2, 87):  # 87 = slides len
    slide = prs.slides[i]
    number = table_shapes[i - 2]
    shape = slide.shapes[number]
    table = shape.table

    count_subject = 0
    for item in range(1, len(list_top_10)):
        if list_top_10[item][0].upper() == title_text[i - 1]:
            count_item = 1
        else:
            count_item = 0
        count_subject += count_item
    count_subject = count_subject

    for t1 in range(1, count_subject + 1):
        for t2 in range(0, 4):
            table.cell(t1,
                       t2).text = list_top_10[t1 + count_subject_count][t2 + 2]
            for paragraph in table.cell(t1, t2).text_frame.paragraphs:
                for run in paragraph.runs:
                    run.font.size = Pt(9)

    count_subject_count += count_subject

In [15]:
prs.save('Презентация за ' + report_month + '.pptx')